In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("/content/healthcare_dataset.csv")

In [ ]:
df = df.drop(columns=["Name", "Date of Admission", "Discharge Date", "Doctor", "Hospital"])
X = df.drop("Test Results", axis=1)
y = df["Test Results"]

In [ ]:
categorical_cols = X.select_dtypes(include="object").columns.tolist()
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

In [ ]:
encoders = {col: LabelEncoder().fit(X[col]) for col in categorical_cols}
for col in categorical_cols:
    X[col] = encoders[col].transform(X[col])

In [ ]:
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

In [ ]:
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=target_encoder.classes_)

In [ ]:
comparison = pd.DataFrame({
    'Actual': target_encoder.inverse_transform(y_test),
    'Predicted': target_encoder.inverse_transform(y_pred)
})

In [ ]:
print(comparison.head(10))

         Actual     Predicted
0      Abnormal  Inconclusive
1      Abnormal        Normal
2      Abnormal        Normal
3  Inconclusive      Abnormal
4      Abnormal      Abnormal
5      Abnormal  Inconclusive
6      Abnormal  Inconclusive
7      Abnormal  Inconclusive
8        Normal      Abnormal
9        Normal        Normal


In [ ]:
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 32.82%


In [ ]:
import pickle

In [ ]:
with open("healthcare_model.pkl", "wb") as f:
    pickle.dump({
        "model": model,
        "features": X.columns.tolist(),
        "encoders": encoders,
        "scaler": scaler,
        "target_encoder": target_encoder,
        "numerical_cols": numerical_cols,
        "categorical_cols": categorical_cols
    }, f)

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.1 MB/s eta 0:00:00


In [ ]:
def predict(age, gender, blood_type, condition, insurance, bill, room, adm_type, medication):
    row = pd.DataFrame({
        "Age": [age],
        "Gender": [encoders["Gender"].transform([gender])[0]],
        "Blood Type": [encoders["Blood Type"].transform([blood_type])[0]],
        "Medical Condition": [encoders["Medical Condition"].transform([condition])[0]],
        "Insurance Provider": [encoders["Insurance Provider"].transform([insurance])[0]],
        "Billing Amount": [bill],
        "Room Number": [room],
        "Admission Type": [encoders["Admission Type"].transform([adm_type])[0]],
        "Medication": [encoders["Medication"].transform([medication])[0]]
    })
    row[numerical_cols] = scaler.transform(row[numerical_cols])
    result = model.predict(row)[0]
    return target_encoder.inverse_transform([result])[0]

In [ ]:
import gradio as gr

In [ ]:
gr.Interface(
    fn=predict,
    inputs=[
        gr.Number(label="Age"),
        gr.Dropdown(choices=encoders["Gender"].classes_.tolist(), label="Gender"),
        gr.Dropdown(choices=encoders["Blood Type"].classes_.tolist(), label="Blood Type"),
        gr.Dropdown(choices=encoders["Medical Condition"].classes_.tolist(), label="Medical Condition"),
        gr.Dropdown(choices=encoders["Insurance Provider"].classes_.tolist(), label="Insurance Provider"),
        gr.Number(label="Billing Amount"),
        gr.Number(label="Room Number"),
        gr.Dropdown(choices=encoders["Admission Type"].classes_.tolist(), label="Admission Type"),
        gr.Dropdown(choices=encoders["Medication"].classes_.tolist(), label="Medication")
    ],
    outputs=gr.Text(label="Predicted Test Result"),
    title="Healthcare Test Result Predictor"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://99cec98c5f8745bd61.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
